# Installation des dépendances 

In [18]:
import getpass
import os


def _set_env(key: str):
    #if key not in os.environ:
        os.environ[key] = getpass.getpass(f"{key}:")


_set_env("MISTRAL_API_KEY") # https://console.mistral.ai/home 

# Chargement des documents

Dans LangChain, on charge les documents avec la classe `DocumentLoader`.

Il existe plusieurs types de document loaders en fonction du type de document à charger :

- `TextLoader` : pour charger un fichier texte
- `CSVLoader` : pour charger un fichier CSV
- `JSONLoader` : pour charger un fichier JSON
- `WebBaseLoader` : pour charger un document web
- `UnstructuredMarkdownLoader` : pour charger un fichier Markdown

**Votre objectif** : charger les documents du dossier `docs`

In [ ]:
from langchain_community.document_loaders import 


# Chargement des documents CSV
csv_loader = 
csv_docs = 

# Chargement du site web https://www.diagngrow.com/
url_loader = 
web_docs = 


# Chargement de tous les documents Markdown du dossier docs/
md_loader = 
md_docs = 

# Combinaison des documents
all_docs = md_docs + csv_docs + web_docs

print(f"Documents Markdown chargés: {len(md_docs)}")
print(f"Documents CSV chargés: {len(csv_docs)}")
print(f"Documents web chargés: {len(web_docs)}")
print(f"Total documents: {len(all_docs)}")

Documents Markdown chargés: 4
Documents CSV chargés: 7
Documents web chargés: 1
Total documents: 12


# Découpage des documentsen Chunks

Dans langchain, on utilise des "splitters" pour découper les documents en chunks.

Il existe plusieurs types de splitters :
- `CharacterTextSplitter` : découpe en fonction du nombre de caractères
- `TokenTextSplitter` : découpe en fonction du nombre de tokens
- `RecursiveCharacterTextSplitter` : découpe en fonction du nombre de caractères, mais permet d'ajouter des chevauchements entre les chunks

In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

# Instanciation et paramétrage du splitter 
# (taille d'un chunk = 1000 caractères, chevauchement = 200 caractères)
text_splitter = 

# Découpage des documents dans "all_docs"
chunks = 

print(f"Total chunks après découpage: {len(chunks)}")
print("Exemple de chunk : ")
print(chunks[0])

Total chunks après découpage: 37
Exemple de chunk : 
page_content='Résultats Financiers 2025 - Diag n'Grow

Rapport Annuel - Exercice clos le 31 décembre 2025

Synthèse de l'Exercice

L'année 2025 a été marquée par une croissance soutenue de l'activité de Diag n'Grow, confirmant la pertinence de notre positionnement sur le marché de la valorisation des actifs immatériels.

Chiffres Clés

Indicateur 2025 2024 Évolution Chiffre d'affaires 892 000 € 620 000 € +43,9% Marge brute 606 560 € 408 200 € +48,6% Taux de marge brute 68% 65,8% +2,2 pts EBITDA 134 000 € 78 000 € +71,8% Résultat d'exploitation 112 000 € 58 000 € +93,1% Résultat net 89 600 € 43 500 € +106%

Compte de Résultat Détaillé

Produits d'exploitation

Poste Montant Prestations d'audit logiciels 312 200 € Valorisation de brevets 267 400 € Audit de marques 178 600 € Missions de due diligence 133 800 € Total Chiffre d'Affaires 892 000 €

Charges d'exploitation' metadata={'source': 'docs/resultats_financiers_2025.md'}


# Vectorisation des chunks

On a besoin de 2 éléments dans cette partie :

1. Modèle de vectorisation

On va utiliser un modèle de vectorisation de texte en utilisant un modèle disponible sur HuggingFace. Par exemple, on peut utiliser un modèle d'embedding de jinaai mais il en existe beaucoup d'autres.

2. Base de données vectorielle

On va utiliser une base de données vectorielle qui s'appelle ChromaDB.
Une base de données prends en argument un modèle de vectorisation et des chunks.

In [ ]:
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_chroma import Chroma

# Création du modèle d'embeddings 
embeddings = 

# Stockage des embeddings dans ChromaDB dans un dossier local "chroma_db"
vectorstore = 

`torch_dtype` is deprecated! Use `dtype` instead!
`torch_dtype` is deprecated! Use `dtype` instead!


# Création du prompt "augmenté" et appel du LLM

Nous allons générer automatiquement un prompt en fonction de la question posée par l'utilisateur : 


Le prompt système sera composé des chunks les plus proches entre la question et les chunks dans ChromaDB : `Tu es un assistant personnalisé. Pour répondre à la question de l'utilisateur, utilise le contexte suivant : ........`


Le prompt utilisateur sera la question posée par l'utilisateur.


Ensuite, on initialise le LLM avec le prompt système et le prompt utilisateur puis on l'invoque pour qu'il réponde à la question de l'utilisateur.

In [ ]:


def ask_to_rag(prompt):
    # recherche des documents similaires dans le vectorstore
    retrieved_docs =

    # Conversion des documents en texte
    docs_content =

    #Création des messages (une liste)
    # chaque message est un tuple (type, contenu)
    # On a besoin du type "system" avec les instructions et documents récuperés
    # et du type "human" avec la question de l'utilisateur
   
    messages = 

    # Init du LLM (Mistral)
    llm = 

    # Appel du LLM avec les messages (prompt system + prompt humain)
    ai_msg = 


    print("Documents utilisés : ")
    print(retrieved_docs)

    print(f"\n Réponse : \n {ai_msg.content}")


In [ ]:
ask_to_rag("Qui est le directeur technique de diag n'grow ?")

Documents utilisés : 
[Document(id='6c31a921-ae9a-4f53-9f38-dd32b8990305', metadata={'source': 'docs/avis_clients.md'}, page_content='Note : 5/5 | Mission réalisée en 3 semaines\n\nMarie-Claire Dubois - Directrice M&A, Capital Innovation Fund\n\nDue Diligence - Octobre 2025\n\n"Nous faisons appel à Diag n\'Grow pour toutes nos due diligences sur les actifs immatériels. Leur rigueur méthodologique et la qualité de leurs livrables sont irréprochables. Laurence et Pierre comprennent parfaitement les enjeux des investisseurs. C\'est devenu un partenaire incontournable pour notre fonds."\n\nNote : 5/5 | 8 missions réalisées en 2025\n\nJean-Philippe Moreau - PDG, Groupe Industriel Martin\n\nValorisation de marques - Juillet 2025\n\n"Notre groupe possède 5 marques historiques que nous n\'avions jamais fait évaluer. Diag n\'Grow nous a fourni une valorisation précise et argumentée qui nous a permis de les inscrire à notre bilan. Le travail de Pedro sur l\'analyse marketing était particulièreme

In [23]:
ask_to_rag("Pourquoi François Petit a fait appel à Diag n'Grow ?")

Documents utilisés : 
[Document(id='8e22d4f0-c493-4ffb-8055-0428f92f4d15', metadata={'source': 'docs/business_plan_2025_2027.md'}, page_content="Conclusion\n\nDiag n'Grow dispose de tous les atouts pour devenir un acteur majeur de la valorisation des actifs immatériels en France. Notre équipe expérimentée, notre approche innovante et notre stratégie de croissance maîtrisée nous positionnent idéalement pour atteindre nos objectifs ambitieux à horizon 2027.\n\nContact : contact@diagngrow.fr"), Document(id='6c31a921-ae9a-4f53-9f38-dd32b8990305', metadata={'source': 'docs/avis_clients.md'}, page_content='Note : 5/5 | Mission réalisée en 3 semaines\n\nMarie-Claire Dubois - Directrice M&A, Capital Innovation Fund\n\nDue Diligence - Octobre 2025\n\n"Nous faisons appel à Diag n\'Grow pour toutes nos due diligences sur les actifs immatériels. Leur rigueur méthodologique et la qualité de leurs livrables sont irréprochables. Laurence et Pierre comprennent parfaitement les enjeux des investisseurs